# RDTools viewer examples

Author: Xiaorui Dong

This is a show case of how you can use the viewers (use py3Dmol and 3Dmol.js as backend) in RDTools to visualize and interact with the RDKit Molecules.

A general guidance of how to interact with the viewer:
| Movement | Mouse Input |
|----------|-------------|
| Rotation | Primary Mouse Button |
| Translation | Middle Mouse Button or Ctrl+Primary |
| Zoom | Scroll Wheel or Second Mouse Button or Shift+Primary |
| Slab | Ctrl+Second |

In [ ]:
from rdkit import Chem

from rdtools.conversion.smiles import mol_from_smiles
from rdtools.view import (
    mol_viewer, mol_animation, ts_viewer,
    conformer_viewer, conformer_animation, interactive_conformer_viewer,
    freq_viewer, merge_xyz_dxdydz,
    reaction_viewer,
    grid_viewer,
)

%load_ext autoreload
%autoreload 2

## 1. Static structure viewer

### 1.1 `mol_viewer`
`mol_viewer` is used to create a 3D viewer of a single conformer of the molecule. The only mandatory input is the mol object.

Create a molecule object and embed a few conformers

In [ ]:
smi = "C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC=C4[C@@]3(CC[C@@H](C4)O)C)C"
mol = mol_from_smiles(smi)  # This is similar to Chem.MolFromSmiles(smi) but with more handy options
Chem.AllChem.EmbedMultipleConfs(mol, 20);

Optional arguments
- `conf_id`: provide an integer to inspect other conformers other than the first one (`0` by default)
- `viewer_size`: `(400, 400)` by default
- `atom_index`: show atom indexes. `True` to persistently show the indexes, `False` to turn it off and only showing the index when hovering the mouse on a specific atom

In [ ]:
mol_viewer(mol, conf_id=10, atom_index=False, viewer_size=(800, 400))

### 1.2 `conformer_viewer`

`conformer_viewer` is used to create a 3D viewer of multiple conformers of the molecule. You have the freedom to choose whichever conformers to include and whichever conformers to highlight.
It is suggested aligning conformers first to have a better visualization effect.

In [ ]:
# Align conformers, and only align based on heavy atoms
Chem.rdMolAlign.AlignMolConformers(mol, atomIds=[atom.GetIdx() for atom in mol.GetAtoms()])

Optional arguments:
- `conf_ids`: provide a list of conformer ids to display. When not providing, it will display all of the conformers.
- `highlight_ids`: provide a list of conformer ids to indicate which ones to highlight. By default, it will highlight all conformers.
- `conf_opacity`: a float number between 0 to 1 to indicate the opacity for conformers that are not highlighted.
- `viewer_size`: `(400, 400)` by default
- `style_spec`: you can also change style specs. The default is spheres and sticks. To only use sticks, try `{'stick': {'radius': 0.25}}`.

In [ ]:
conformer_viewer(
    mol,
    conf_ids=list(range(10)),  # showing the first 10
    highlight_ids=[0],
    conf_opacity=0.4,
    style_spec={"stick": {"radius": 0.25}},
)

### 1.3 `ts_viewer`

`ts_viewer` provides a special treatment to the bonds that are formed or broken compared to `mol_viewer`. Here we use a naive H transfer example to illustrate the idea.

In [ ]:
from rdtools.conversion import mol_from_xyz

xyz = \
    """4
    
    H      2.3 0.0 0.0
    C      1.2 0.0 0.0
    H     -0.5 0.0 0.0
    H     -1.8 0.0 0.0"""

mol = mol_from_xyz(xyz)  # This function creates a molecule object from the xyz

Arguments:
- mol: The mol has the transition state geometry embedded.
- `broken_bonds`: a list of atom pair indexes indicate the broken bonds.
- `formed_bonds`: a list of atom pair indexes indicate the formed bonds.

You can also change the color and the width of the changing bonds. Besides, you can use the optional arguments that `mol_viewer` uses to further tune the illustration.

In [ ]:
ts_viewer(
    mol,
    conf_id=0,
    broken_bonds=[(1, 2)],
    formed_bonds=[(2, 3)],
    broken_bond_color='red',
    formed_bond_color='green',
    broken_bond_width=0.05,
    formed_bond_width=0.05,
)

### 1.4 `grid_viewer`

You can use grid_viewer to show multiple molecules in a single view.

Create a list of molecules and embed a conformer for each of the molecule

In [ ]:
smis = ["C"+ "C" * i + "O" for i in range(4)]
mols = [mol_from_smiles(smi) for smi in smis]
[Chem.AllChem.EmbedMolecule(mol) for mol in mols];

Create a grid viewer with four sub-viewers.
Arguments:
- `viewer_grid`: a tuple to indicate the number of rows and the number of columns.
- `viewer_size` (optional): the size of the grid

In [ ]:
grid = grid_viewer(viewer_grid=(1, 4), viewer_size=(1200, 400))

Create each sub-viewer with `mol_viewer`. other viewer can be used as well.

In [ ]:
for i in range(4):
    mol_viewer(mols[i], viewer=grid, viewer_loc=(0, i))
grid

### 1.5 `reaction_viewer`

`reaction_viewer` combines a few viewers we just saw together, so that we can visualize reactant, products, as well as TS (if provided) simultaneously.

In [ ]:
from rdtools.conversion import mol_from_xyz  # This function creates a molecule object from the xyz

xyz_r = "3\n\nH 1.0 0.0 0.0\nH -0.25 0.0 0.0\nH -1.0 0.0 0.0"
xyz_ts = "3\n\nH 1.0 0.0 0.0\nH 0.0 0.0 0.0\nH -1.0 0.0 0.0"
xyz_p = "3\n\nH 1.0 0.0 0.0\nH 0.25 0.0 0.0\nH -1.0 0.0 0.0"

r_mol, p_mol, ts_mol = map(mol_from_xyz, [xyz_r, xyz_p, xyz_ts])

Arguments:
- `r_mol`, `p_mol` are required as of the reactant and the product of the molecule
- `ts_mol` is optional. If you don't provide `ts_mol`, then only `r_mol` and `p_mol` will be displayed
- `alignment`: you can choose to visualize them `horizontal`ly or `vertical`ly.
- `linked`: If linked, all molecules will move simultanously. Otherwise, you will just move the current molecule

In [ ]:
reaction_viewer(r_mol, p_mol, ts_mol)

## 2. Animation Viewer
! If you are using Jupyter inside VS Code, the frame rate of the animation can be less stable 

### 2.1 `mol_animation`

Combine conformation of different molecules and put them into a short animation. 

Below is the same example of a few alcohol molecules, but now they are shown in a short animation

In [ ]:
smis = ["C" + "C" * i + "O" for i in range(4)]
mols = [mol_from_smiles(smi) for smi in smis]
[Chem.AllChem.EmbedMolecule(mol) for mol in mols];

Arguments: 
- `mols`: molecules should be passed as a sequence, e.g., a list
- `conf_ids`: Optional. if some molecules containing more than one conformer, you may provide a list indicating the conformer id to use for each molecule. It should has the same length as `mols`. Otherwise, just pass `None` to it.
- `interval`: Optional. The time molecule stays before showing the next molecules in the unit of ms. Defaults to `1000` (1 second).

In [ ]:
mol_animation(mols, conf_ids=None, interval=1000)

### 2.2 `conformer_animation`

`conformer_animation` is very similar to `mol_animation` but insteading is showing conformer structures of a single molecule. This can be very useful visualizing results from PES scan or trajectory calculation. We are re-using the cholesterol example as an naive illustration.

In [ ]:
smi = "C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC=C4[C@@]3(CC[C@@H](C4)O)C)C"
mol = mol_from_smiles(smi)  # This is similar to Chem.MolFromSmiles(smi) but with more handy options
Chem.AllChem.EmbedMultipleConfs(mol, 20);
# Align conformers, and only align based on heavy atoms
Chem.rdMolAlign.AlignMolConformers(mol, atomIds=[atom.GetIdx() for atom in mol.GetAtoms()])

Arguments: 
- `mol`: Molecule.
- `conf_ids`: Optional. You may provide a list indicating the conformer id to use if not using all conformers. It should has the same length as `mols`. Otherwise, just pass `None` to it.
- `interval`: Optional. The time molecule stays before showing the next molecules in the unit of ms. Defaults to `1000` (1 second).

In [ ]:
conformer_animation(mol, conf_ids=list(range(20))[::2], interval=1000)

### 2.3 `freq_viewer`

If you have the vibrational displacement matrix of a molecule, you can also visualize its vibrational motion as well. Currently, it only supports a special xyz format where dx, dy, dz of an atom is attached after the atom position. We provide a helper function (`merge_xyz_dxdydz`) to merge the xyz string with the displacement matrix. The following is a naive example of hydrogen's vibration.

In [ ]:
xyz = \
    """2
    [Geometry 1]
    H    0.3720870000   0.0000000000    0.0000000000
    H   -0.3720870000   0.0000000000    0.0000000000"""

dxdydz = [
    [  0.71,   0.  ,  0.  ,],
    [ -0.71,   0.  ,  0.  ,],
]
xyz_input = merge_xyz_dxdydz(xyz, dxdydz)
print(xyz_input)

Arguments:
- xyz_input: the input similar to the above example
- frames (optional): Number of frames to be created. a larger number results in a better animation resolution and a slower motion during display. Defaults to `10`.
- amplitude (optional): Controls the magnitude of the motion. Defaults to `1.0`.

In [ ]:
freq_viewer(xyz_input, frames=20, amplitude=1.0)

## 3. Interactive Viewer
The interactive viewer is in between the static viewer and the animation viewer, where the user has the freedom to choose when to stop and continue, when visualizing multiple structures.

In order to use interactive viewers, please ensure that `ipywidgets` is installed.

### 3.1 `interactive_conformer_viewer`

In [ ]:
smi = "C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC=C4[C@@]3(CC[C@@H](C4)O)C)C"
mol = mol_from_smiles(smi)  # This is similar to Chem.MolFromSmiles(smi) but with more handy options
Chem.AllChem.EmbedMultipleConfs(mol, 20);
# Align conformers, and only align based on heavy atoms
Chem.rdMolAlign.AlignMolConformers(mol, atomIds=[atom.GetIdx() for atom in mol.GetAtoms()])

Arguments:
- `mol`: can be a Mol or a list of molecules. If a list is provided, the first conformer will be displayed.

Optional arguments are the same as `mol_viewer`.

Don't worry if you got a warning immediately running the following cell. You can move the slider, and the warning should then go away.

In [ ]:
interactive_conformer_viewer(mol, atom_index=False, viewer_size=(1000, 400))